In [ ]:
"""
=================================
Gaussian Mixture Model Selection
=================================

This example shows that model selection can be performed with
Gaussian Mixture Models using information-theoretic criteria (BIC).
Model selection concerns both the covariance type
and the number of components in the model.
In that case, AIC also provides the right result (not shown to save time),
but BIC is better suited if the problem is to identify the right model.
Unlike Bayesian procedures, such inferences are prior-free.

In that case, the model with 2 components and full covariance
(which corresponds to the true generative model) is selected.
"""
print(__doc__)

import itertools

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture

# Number of samples per component
n_samples = 50

# Generate random sample, two components
np.random.seed(0)
C = np.array([[0., -0.1], [1.7, .4]])
X = np.r_[np.dot(np.random.randn(n_samples, 2), C),
          .7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]




# number of demos
ndemo = 2

# number of periods
nperiod = np.pi/4

# motor specific
pmin = -60
pmax = 60

# record parameter
framerate = 10
tmin = 0
tmax = 20

# noise parameter
noise_amplitude =  0.2*(pmax-pmin) #30
noise_phase = np.pi/6#1.3
noise_time = 0.7#0.5


datas = Sanitize_records_for_gmm()


def _rand():
    return 2*(np.random.random(1)-1)

t=np.linspace(tmin,tmax,(tmax-tmin)*framerate)
y = ((pmax-pmin)/2)*np.sin(t*2*np.pi*nperiod/tmax) + (pmax+pmin)/2
datas.add_record(t,y)
mot_data = []
for i in xrange(ndemo):
    noisy_t =np.zeros(len(t))
    for n in range(len(t)):
        noisy_t[n] = t[n]+noise_time*np.random.random()
    y_noisy = (noise_amplitude*_rand())+ ((pmax-pmin)/2)*np.sin((noisy_t*2*np.pi*nperiod/tmax)+noise_phase*_rand()) + (pmax+pmin)/2
    mot_data.append(y_noisy)
    datas.add_record(noisy_t,y_noisy)

    #plt.plot(t,y_noisy)

# mot = 
#f, axes = plt.subplots(ndemo, 1)

#for i in range(ndemo):
#    axes[i].plot(mot_data[i])
X = datas.to_array()



lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a mixture of Gaussians with EM
        gmm = mixture.GMM(n_components=n_components, covariance_type=cv_type)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['k', 'r', 'g', 'b', 'c', 'm', 'y'])
clf = best_gmm
bars = []

# Plot the BIC scores
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

# Plot the winner
splot = plt.subplot(2, 1, 2)
Y_ = clf.predict(X)
for i, (mean, covar, color) in enumerate(zip(clf.means_, clf.covars_,
                                             color_iter)):
    v, w = linalg.eigh(covar)
    if not np.any(Y_ == i):
        continue
    plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180 * angle / np.pi  # convert to degrees
    v *= 4
    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180 + angle, color=color)
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)

plt.xlim(-10, 10)
plt.ylim(-3, 6)
plt.xticks(())
plt.yticks(())
plt.title('Selected GMM: full model, 2 components')
plt.subplots_adjust(hspace=.35, bottom=.02)
plt.show()
print X


In [3]:

class Sanitize_records_for_gmm(object):
    """ Merge records data for gmm  
    """
    def __init__(self,dimension=1):
        self.times = []
        self.dimension = dimension
        self.positions = []
        self._X = []
        self.gmm = None
        #self.__x_updated__ = False
    def add_record(self,times,positions):
        if len(times) != len(positions):
            raise Exception("list must have same dimensions")
        # Dirty Sort
        X1 = [i for i in zip (times,positions)]
        X2 = [i for i in zip (self.times,self.positions)]
        X =  sorted(X1+X2)
        self.positions = []
        self.times = []
        for i in X:
            self.times.append(i[0])
            self.positions.append(i[1])
    #def _gen_X(self):
    #    self._XX = np.array([self.times,self.positions]).transpose()
    
    def to_array(self):
        self._X = np.array([self.times,self.positions]).transpose()
        return self._X
    
    def plot(self,ax=None):
        X = self.to_array()
        if ax is None:
            plt.scatter(X[:,0],X[:,1])
        else :
            ax.scatter(X[:,0],X[:,1])
    
    def gen_gmm(self):
        X = self.to_array()
        title = 'Expectation-maximization'
        self.gmm = mixture.GMM(n_components=4, covariance_type='full', n_iter=100)
        self.gmm.fit(X)
        return self.gmm
    
    def plot_gmm(self,ax=None):
        color_iter = itertools.cycle(['r', 'g', 'b', 'c', 'm'])
        self.gen_gmm()
        Y_ = self.gmm.predict(self._X)
        if ax is None:
            ax = plt.subplot(1,1,1)

        for i, (mean, covar, color) in enumerate(zip(
                self.gmm.means_, self.gmm._get_covars(), color_iter)):
                ## valeur propre, vecteur propre
                v, w = linalg.eigh(covar)
                u = w[0] / linalg.norm(w[0])
                # as the DP will not use every component it has access to
                # unless it needs it, we shouldn't plot the redundant
                # components.
                if not np.any(Y_ == i):
                    continue
                ax.scatter(self._X[Y_ == i, 0], self._X[Y_ == i, 1], .8, color=color)

                # Plot an ellipse to show the Gaussian component
                angle = np.arctan(u[1] / u[0])
                angle = 180 * angle / np.pi  # convert to degrees
                ell = mpl.patches.Ellipse(mean, v[0], v[1], 180 + angle, color=color)
                ell.set_clip_box(ax.bbox)
                ell.set_alpha(0.5)
                ax.add_artist(ell)
    
    
        
    
        
    
    